# 증권사 API 연결과 매매하기

## 접근 토큰 및 해시키 발급하기

In [1]:
import requests
import json
import keyring

# key
app_key = keyring.get_password('koreainvest_mock_app_key', 'ahn283')
app_secret = keyring.get_password('koreainvest_mock_app_secret', 'ahn283')

# base url
url_base = 'https://openapivts.koreainvestment.com:29443'        # 모의투자

# information
headers = {"content-type": "application/json"}
path = "oauth2/tokenP"
body = {
    "grant_type": "client_credentials",
    "appkey": app_key,
    "appsecret": app_secret
}
url = f"{url_base}/{path}"
print(url)

https://openapivts.koreainvestment.com:29443/oauth2/tokenP


In [2]:
# 위 url을 body에 전송하면 접근 토근을 받을 수 있다.
# dumps() 함수를 통해 딕셔너리를 json 문자열로 변환하여 전송
res = requests.post(url, headers=headers, data=json.dumps(body))
access_token = res.json()['access_token']

### 해시키 발급받기

In [3]:
def hashkey(datas):
    path = 'uapi/hashkey'
    url = f'{url_base}/{path}'
    headers = {
        'content-type': 'application/json',
        'appKey': app_key,
        'appSecret': app_secret
    }
    res = requests.post(url, headers=headers, data=json.dumps(datas))
    hashkey = res.json()['HASH']
    
    return hashkey

## 주식 현재가 시세 조회하기

In [4]:
# 삼성전자

path = 'uapi/domestic-stock/v1/quotations/inquire-price'
url = f'{url_base}/{path}'

headers = {
    'Content-type' : 'application/json',
    'authorization' : f'Bearer {access_token}',
    'appKey' : app_key,
    'appSecret' : app_secret,
    # tr_ud는 거래ID에 해당하는 부분으로써 주식현재가 시세에 해당하는 FHKST0101010을 입력한다.
    'tr_id' : 'FHKST01010100'
}
# fid_cond_mrkt_div_code는 시장 분류코드며, 주식에 해당하는 J를 입력하고 fid_input_iscd에는 삼성전자 티커를 입력
params = {'fid_cond_mrkt_div_code': 'J', 'fid_input_iscd': '005930'}

res = requests.get(url, headers=headers, params=params)
res.json()['output']['stck_prpr']

'72600'

## 주식 주문하기

### 매수주문

In [5]:
path = '/uapi/domestic-stock/v1/trading/order-cash'
url = f'{url_base}/{path}'

data = {
    'CANO': '50099655',     # 계좌번호 앞 8자리
    'ACNT_PRDT_CD': '01',   # 계좌번호 뒤 2자리
    'PDNO': '005930',       # 종목코드
    'ORD_DVSN': "01",       # 주문 방법 : 01-시장가 주문
    'ORD_QTY': "10",        # 주문   
    'ORD_UNPR': "0"         # 주문 단가 (시장가의 경우 0)    
}

headers = {
    "Content-Type": "application/json",
    "authorization": f"Bearer {access_token}",
    "appKey": app_key,
    "appSecret": app_secret,
    "tr_id": "VTTC0802U",               # 주식 현금 매수 주문 tr_id
    "custtype": "P",
    "hashkey": hashkey(data)            # 주문 보안을 위해 해시키를 발급받아 입력
}

# post() 함수를 통해 요청
res = requests.post(url, headers=headers, data=json.dumps(data))
res.json()

{'rt_cd': '1', 'msg_cd': '40580000', 'msg1': '모의투자 장종료 입니다.'}

### 정정 주문

In [6]:
path = '/uapi/domestic-stock/v1/trading/order-cash'
url = f'{url_base}/{path}'

data = {
    'CANO': '50099655',     # 계좌번호 앞 8자리
    'ACNT_PRDT_CD': '01',   # 계좌번호 뒤 2자리
    'PDNO': '005930',       # 종목코드
    'ORD_DVSN': "00",       # 주문 방법
    'ORD_QTY': "10",        # 주문 수량
    'ORD_UNPR': "50000",    # 주문 단가 (시장가의 경우 0)
}

headers = {
    'Content-Tye': 'application/json',
    'authorization': f'Bearer {access_token}',
    'appKey': app_key,
    'appSecret': app_secret,
    'tr_id': 'VTTC0802U',
    'custtype': 'P',
    'hashkey': hashkey(data)
}

res = requests.post(url, headers=headers, data=json.dumps(data))
res.json()

{'rt_cd': '1', 'msg_cd': '40580000', 'msg1': '모의투자 장종료 입니다.'}

In [8]:
# 한국거래소전송주문 조직번호과 주문번호
# res에서 구할 수 있음
KRX_FWDG_ORGNO = res.json()['output']['KRX_FWDG_ORD_ORGNO']     # 한국거래소전송주문조직번호
ODNO = res.json()['output']['ODNO']         # 주문번호

In [ ]:
# 미체결된 주문을 최유리 지정가로 변경 정정 주문
path = 'uapi/domestic-stock/v1/trading/order-rvsecncl'
url = f'{url_base}/{path}'

data = {
    'CANO': '50099655',     # 계좌번호 앞 8자리
    'ACNT_PRDT_CD': '01',   # 계좌번호 뒤 2자리
    'KRX_FWDG_ORD_ORGNO': KRX_FWDG_ORGNO,   # 한국거래소전송주문 조직번호
    'ORGN_ODNO': ODNO,      # 주문번호
    'ORD_DVSN': '03',       # 주문 방법
    'RVSE_CNCL_DVSN_CD': '01',  # 정정 (취소는 02)
    'ORD_QTY': '10',        # 주문 수량
    'ORD_UNPR': '0',        # 주문 단가 (시장가의 경우 0)
    'QTY_ALL_ORD_YN': 'Y'   # 잔량 전부 (잔량 일부는 N)    
}

headers = {
    'Content-Type': 'application/json',
    'authorization': f'Bearer {access_token}',
    'appKey': app_key,
    'appSecret': app_secret,
    'tr_id': 'VTTC0803U',
    'custtype': 'P',
    'hashkey': hashkey(data)
}

res = requests.post(url, headers=headers, data=json.dump(data))
res.json()

### 매도 주문

In [ ]:
# 삼성전자 10주 시장가 매도
path = '/uapi/domestic-stock/v1/trading/order-cash'
url = f'{url_base}/{path}'

data = {
    'CANO': '50099655',     # 계좌번호 앞 8자리
    'ACNT_PRDT_CD': '01',   # 계좌번호 뒤 2자리
    'PDNO': '005930',       # 종목코드
    'ORD_DVSN': '01',       # 주문 방법
    'ORD_QTY': '10',        # 주문 수량
    'ORD_UNPR': '0',        # 주문 단가 (시장가의 경우 0)    
}

headers = {
    'Content-Type': 'application/json',
    'authorization': f'Bearer {access_token}',
    'appKey': app_key,
    'appSecret': app_secret,
    'tr_id': 'VTTC0801U',
    'custtype': 'P',
    'hashkey': hashkey(data)
}

res = requests.post(url, headers=headers, data=json.dumps(data))

## 주식 잔고조회

In [10]:
path = '/uapi/domestic-stock/v1/trading/inquire-balance'
url = f'{url_base}/{path}'

headers = {
    'Content-Typoe': 'application/json',
    'authorization': f'Bearer {access_token}',
    'appKey': app_key,
    'appSecret': app_secret,
    'tr_id': 'VTTC8434R'
}

params = {
    'CANO': '50099655',     # 계좌번호 앞 8자리
    'ACNT_PRDT_CD': '01',   # 계좌번호 뒤 2자리
    'AFHR_FLPR_YN': 'N',    # 시간외단일가여부
    'OFL_YN': '',           # 공란
    'INQR_DVSN': '01',      # 조회구분
    'UNPR_DVSN': '01',      # 단가구분
    'FUND_STTL_ICLD_YN': 'N',   # 펀드결제분포함여부
    'FNCG_AMT_AUTO_RDPT_YN': 'N',    # 융자금액자동상환여부
    'PRCS_DVSN': '00',      # 처리구분 (00: 전일매매포함)
    'CTX_AREA_FK100': '',   # 연속조회검색조건
    'CTX_AREA_NK100': '',   # 연속조회키    
}

# get 함수 통해 데이터를 요청
res = requests.get(url, headers=headers, params=params)

In [13]:
# ['output1']에는 보유 종목에 대한 정보
res.json()['output1']

[]

In [14]:
# ['output2']에는 계좌정보가 들어 있다.
res.json()['output2']

[{'dnca_tot_amt': '500000000',
  'nxdy_excc_amt': '500000000',
  'prvs_rcdl_excc_amt': '500000000',
  'cma_evlu_amt': '0',
  'bfdy_buy_amt': '0',
  'thdt_buy_amt': '0',
  'nxdy_auto_rdpt_amt': '0',
  'bfdy_sll_amt': '0',
  'thdt_sll_amt': '0',
  'd2_auto_rdpt_amt': '0',
  'bfdy_tlex_amt': '0',
  'thdt_tlex_amt': '0',
  'tot_loan_amt': '0',
  'scts_evlu_amt': '0',
  'tot_evlu_amt': '500000000',
  'nass_amt': '500000000',
  'fncg_gld_auto_rdpt_yn': '',
  'pchs_amt_smtl_amt': '0',
  'evlu_amt_smtl_amt': '0',
  'evlu_pfls_smtl_amt': '0',
  'tot_stln_slng_chgs': '0',
  'bfdy_tot_asst_evlu_amt': '500000000',
  'asst_icdc_amt': '0',
  'asst_icdc_erng_rt': '0.00000000'}]

In [15]:
# 티커 종목명, 보유수량, 매입평균가격을 데이터 프레임으로 변환
import pandas as pd

ap = pd.DataFrame.from_records(res.json()['output1'])
ap

""


## 스케쥴링

In [2]:
import datetime

def job():
    print(datetime.datetime.now().strftime('%H:%M:%S'))
    print("=================")

In [3]:
# 매 3초마다 job() 함수 실행

import schedule
schedule.every(3).seconds.do(job)

Every 3 seconds do job() (last run: [never], next run: 2023-12-08 18:41:06)

In [4]:
# 등록된 스케쥴 확인
schedule.get_jobs()

[Every 3 seconds do job() (last run: [never], next run: 2023-12-08 18:41:06)]

In [20]:
# # 함수 실행
# while True:
#     schedule.run_pending()

18:35:23
18:35:26
18:35:29
18:35:32
18:35:35


: 

### 시간 지정하기

In [5]:
# 작업이 실행될 시간과 종료할 시간을 직접 지정

import pandas as pd
from datetime import timedelta

schedule.clear()

startDt = datetime.datetime.now() + timedelta(seconds=60)
endDt = datetime.datetime.now() + timedelta(seconds=80)
# date_range 함수를 통해 startDt와 endDt를 10개 구간으로 나눈다.
time_list = pd.date_range(startDt, endDt, periods=5)

print(time_list)

DatetimeIndex(['2023-12-08 18:42:10.553017', '2023-12-08 18:42:15.553017',
               '2023-12-08 18:42:20.553017', '2023-12-08 18:42:25.553017',
               '2023-12-08 18:42:30.553017'],
              dtype='datetime64[ns]', freq=None)


In [6]:
# 스케쥴러 등록에 필요한 %H:%M:%S 형태로 변환
time_list_sec = [i.strftime('%H-%M-%S') for i in time_list]
time_list_sec

['18-42-10', '18-42-15', '18-42-20', '18-42-25', '18-42-30']

In [7]:
# job() 함수를 수행하도록 스케쥴을 등록
[schedule.every().day.at(i).do(job) for i in time_list_sec]

ScheduleValueError: Invalid time format for a daily job (valid format is HH:MM(:SS)?)

In [8]:
while True:
    schedule.run_pending()
    if datetime.datetime.now() > endDt:
        print('End')
        schedule.clear()
        break

End


## 포트폴리오 리밸런싱